In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P
from mppi.Utilities import Tools
from mppi.Utilities import Constants as Const
import matplotlib.pyplot as plt
import numpy as np
import os, yaml
from IPython.display import Image

In [3]:
omp = 1
mpi = 4 #16
sbatch_options = ['--partition=all12h']

In [4]:
code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo')
#code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo',scheduler='slurm',sbatch_options=sbatch_options)
code.global_options()

Initialize a Yambo calculator with scheduler direct


{'omp': 1,
 'mpi': 4,
 'mpi_run': 'mpirun -np',
 'executable': 'yambo',
 'scheduler': 'direct',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'sbatch_options': [],
 'activate_BeeOND': True,
 'verbose': True}

# GW convergence analysis of the Germanene

We compute the GW corrections of the DFT band structure of the Germanene. 